# **MEDIA ONLINE:**

    - kompas
    - detik.com

## REQUIREMENTS

    - Python 3.9 > latest version
    - Selenium
    - Request
    - Beautifulsoup4
    - pandas
    - time
    - openpyxl

In [1]:
def split_site(teks):

    # Memisahkan teks berdasarkan tanda hubung
    title_split = teks.split("–")

    # Mengambil bagian yang diinginkan dan menghilangkan spasi tambahan
    title = title_split[0].strip()
    
    return title

In [11]:
### READ DATAFRAME
import pandas as pd

filename = "freeport data1.xlsx"

dataframe = pd.read_excel(filename)
print(f"Example Title: {dataframe['Title'][0]}")
dataframe.head()

Example Title: Impian Indonesia Menjadi Raja Baterai Hampir Menjadi Kenyataan, Ini Faktanya! – Cnbcindonesia.com, 31 Juli 2023.


,Title,Content,Result
0,Impian Indonesia Menjadi Raja Baterai Hampir M...,NaN,Kementerian Koordinator Bidang Kemaritiman dan...
1,"Wajib OSS-RBA Bagi Kontraktor di Freeport, Rom...",NaN,Dinas Penanaman Modal dan Pelayanan Terpadu Sa...
2,Freeport Indonesia Serahkan Dua Ton Bantuan Da...,NaN,PT Freeport Indonesia (PTFI) menyerahkan dua t...
3,Hujan Es Juga Melanda Papua pada 2019 dan 2015...,NaN,Kepala Pusat Informasi Perubahan Iklim Badan M...
4,Waasops Kogabwilhan III Minta TNI dari Satgas ...,NaN,Asisten Wakil Operasi Komando Gabungan Wilayah...


In [14]:
"""
Karena pada title terdapat sumber berita (- Cnbcindonesia.com) maka akan dilakukan cleaning dengan melakukan split
seperti pada kode dibawah ini.
"""
dataframe["Title"] = dataframe["Title"].apply(split_site)
print(f"Example Title: {dataframe['Title'][0]}")

Example Title: Impian Indonesia Menjadi Raja Baterai Hampir Menjadi Kenyataan, Ini Faktanya!


In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

"""
Untuk melakukan scraping data, diperlukan beberapa modul yang harus di install seperti pada kode import diatas.
Fungsi dibawah akan mengimplementasikan webdriver menggunakan selenium. search engine yang digunakan adalah Edge. 
Bisa menggunakan Chrome atau search engine lainnya yang didukung oleh selennium. 
alasan tidak melakukan pencarian ke situs media massa online secara langsung karena ketika dilakukan pencarian secara manual hasil pencarian tidak ditemukan. oleh karena itu, solusinya adalah melakukan pencarian ke bing dengan menambahkan spesifik pencarian menggunakan {site:(situs).com}. setelah url tersedia, selanjutnya webdriver akan mendapatkan menggunakan fungsi get yang menerima parameter url.
selebihnya bisa menyesuaikan element yang akan di temukan untuk melakukan pengambilan data.
Untuk mengubah driver ke tab selanjutnya bisa memanfaatkan fungsi window_handles dari selenium.

disclaimer: Fungsi ini digunakan untuk mendapatkan current url dari berita yang digunakan
"""


def get_title_url_kompas(query, site):
    driver = webdriver.Edge()
    
    query = query+ f" site:{site}.com".replace(" ", "+")
    url = f'https://www.bing.com/search?q={query}'
    driver.get(url)
    time.sleep(3)
    
    search_news = driver.find_element(By.XPATH, "//*[@id=\"b_results\"]/li[1]/div[1]/a").click()
    time.sleep(3)
    
    all_handles = driver.window_handles
    driver.switch_to.window(all_handles[1])
    time.sleep(2)
    
    url = driver.find_element(By.XPATH, "/html/head/meta[19]").get_attribute("content")
    
    driver.quit()
    
    return url

In [72]:
## TIDAK DIGUNAKAN (BISA DIGUNAKAN APABILA INGIN MELAKUKAN PENCARIAN PADA FITUR SEARCH KOMPAS)

"""
Untuk melakukan pencarian langsung ke suatu situs. bisa memanfaatkan Keys yang akan melakukan pengiriman inputan.
"""

from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def get_link_kompas_news(title):
    driver = webdriver.Edge()
    url = 'https://www.kompas.com/'
    driver.get(url)
    time.sleep(3)

    title = driver.find_element(by=By.ID, value="search").send_keys(title)
    submt = driver.find_element(by=By.ID, value="search").send_keys(Keys.ENTER)

    time.sleep(2)

    ceks = driver.find_element(by=By.CSS_SELECTOR, value="div.gs-title").get_attribute("outerHTML")

    page_content = BeautifulSoup(ceks, "html.parser")
    title = page_content.find("a", class_="gs-title")
    
    return title["href"]


In [73]:
"""
    Membersihkan konten yang tidak diperlukan dengan menggunakan fungsi decompose.
    lalu return content yang telah dibersihkan
"""

def clean_content(page_content, to_be_erase):
    content = page_content.find("div", class_="read__content") 
    for tag in to_be_erase:
        tag.decompose()
    return content

In [74]:
import requests
from bs4 import BeautifulSoup

"""
Setelah url didapatkan, selanjutnya dilakukan scrapping dengan menggunakan bs4 dan request.
"""

def get_content(url):
    
    response = requests.get(url) # REQUEST URL
    
    page_content = BeautifulSoup(response.content, "html.parser") #Menggunakan BS4 untuk mendapatkan struktur html
    
    erase = page_content(["strong", "i", "iframe"]) # Ini adalah element yang tidak diperlukan dan akan dibersihkan
    content = clean_content(page_content, erase).text.strip() # pangggil clean_content lalu ambil teks setelahnya fungsi strip digunakan untuk membersihkan baris kosong
    
    return content #kembalikan kontent

In [75]:
def main(query):
    
    """
        Main program adalah fungsi untuk mendapatkan url lalu mengambil kontent
    """
    
    site = ["kompas", "detik"]
    url = get_title_url_kompas(query, site[0])
    if url:
        url = url
        time.sleep(2)
    else:
        url = get_title_url_kompas(query, site[1])
        time.sleep(2)
        
    content = get_content(url)
    
    return content

In [76]:
content_list = [] # Untuk menampung konten dalam bentuk list
# for index in range(len(dataframe["Title"])-1):
start = 50 #Mulai indexing pada DataFram (dimulai dari index 0)
finish = 100 #Akhir index digunakan untuk membatasi index

"""
    Start dan finish digunakan untuk membatasi program, namun bisa tidak digunakan apabila device Kuat untuk menjalankan program yang lama
"""

print("GENERATE START...\n")
# for index in range(start, finish): #Mengambil index dari range start sampai finish
    # title = dataframe["Title"].iloc[index] #gunakan dataframe yang diinginkan lalu fungsi iloc untuk mendapatkan baris sesuai index
for title in dataframe["Judul"].iloc[start:finish]:
    """
        Try, Except digunakan untuk mencegah error.
        biasanya kalau url tidak didapatkan akan mendapatkan error pada saat menjalan program mendapatkan kontent karena url yang mau di request.get tidak ada
    """
    
    try:
        results = main(title)
        content_list.append(results)
        print(f"Generate Title {title} Success")

    except:
        content_list.append("")
        print(f"Generate Title {title} Failed")

print("\nGENERATE FINISH...")

GENERATE START...

Generate Title Kemenhub Intensif Lakukan Pengujian LRT Jabodebek Success
Generate Title Uji Coba LRT Jabodebek Makin Intensif Jelang Dioperasikan dengan Penumpang Pekan Depan Failed
Generate Title Segera Beroperasi, Kemenhub Intensif Uji Coba LRT Jabodebek Success
Generate Title Persiapan Beroperasi Komersial, Kemenhub Intensif Lakukan Pengujian terhadap LRT Jabodebek Success
Generate Title Kemenhub Intensif Lakukan Pengujian LRT Jabodebek Success
Generate Title Kemenhub mengintensifkan pengujian LRT Jabodebek Success
Generate Title Kemenhub mengintensifkan pengujian LRT Jabodebek Success
Generate Title Kemenhub Masih Uji LRT Jabodebek, Belum Terbitkan Izin Operasi Success
Generate Title Kabar Baik! Kereta Cepat Jakarta-Bandung Sediakan 600 Tiket Gratis, Siapa Saja Warga Prioritasnya? Success
Generate Title LRT Ditargetkan Beroperasi pada Agustus 2023 Success
Generate Title Kemenhub Sebut Teknologi LRT Jabodebek Lebih Canggih dari MRT Success
Generate Title Kemenhub 

In [77]:
"""
    Simpan pada dataframe yang digunakan dengan spesifik baris menggunakan fungsi
"""

dataframe["Content"].iloc[start:finish] = content_list

In [78]:
"""
    Program untuk menyimpan dataframe kedalam EXCEL
"""

# Specify the Excel file path
excel_file_path = 'Dataset_DKJA.xlsx'

# Create an Excel writer
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    
    dataframe.to_excel(writer, sheet_name="Freeport", index=False)
    print("Success Generate to Excel")

Success Generate to Excel
